In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/satssehgal/MLAPLI/master/bankloan.csv')
df=df.dropna()
df.isna().any()
df=df.drop('Loan_ID', axis=1)
df['LoanAmount']=(df['LoanAmount']*1000).astype(int)
Counter(df['Loan_Status'])

Counter({'N': 148, 'Y': 332})

In [3]:
y_pre = df['Loan_Status']
x_pre = df.drop('Loan_Status', axis=1)
dm_x = pd.get_dummies(x_pre)
dm_y = y_pre.map(dict(Y=1, N=0))
dm_x

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,1.0,4583,1508.0,128000,360.0,1.0,0,1,0,1,1,0,1,0,1,0,0
2,0.0,3000,0.0,66000,360.0,1.0,0,1,0,1,1,0,0,1,0,0,1
3,0.0,2583,2358.0,120000,360.0,1.0,0,1,0,1,0,1,1,0,0,0,1
4,0.0,6000,0.0,141000,360.0,1.0,0,1,1,0,1,0,1,0,0,0,1
5,2.0,5417,4196.0,267000,360.0,1.0,0,1,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,2900,0.0,71000,360.0,1.0,1,0,1,0,1,0,1,0,1,0,0
610,3.0,4106,0.0,40000,180.0,1.0,0,1,0,1,1,0,1,0,1,0,0
611,1.0,8072,240.0,253000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
612,2.0,7583,0.0,187000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1


In [4]:
smote = SMOTE(sampling_strategy='minority')
x1, y = smote.fit_sample(dm_x, dm_y)
sc = MinMaxScaler()
x = sc.fit_transform(x1)

In [5]:
Counter(y)

Counter({0: 332, 1: 332})

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=1234)

In [7]:
rf = RandomForestClassifier(n_estimators = 100)
rf = rf.fit(X_train, y_train)

In [8]:
et = ExtraTreesClassifier(n_estimators = 100)
et = et.fit(X_train, y_train)

In [9]:
mnb = MultinomialNB()
mnb = mnb.fit(X_train, y_train)

In [10]:
lr=LogisticRegression(max_iter=1000)
lr = lr.fit(X_train, y_train)

In [11]:
svc= svm.SVC(kernel='linear',probability=True)
svc = svc.fit(X_train,y_train)

In [12]:
evc=VotingClassifier(estimators=[('mnb', mnb),('lr', lr),('rf', rf),('et', et)],voting='hard')
evc = evc.fit(X_train, y_train)

In [13]:
joblib.dump(rf, "./random_forest.joblib", compress=True)
joblib.dump(et, "./extra_trees.joblib", compress=True)
joblib.dump(mnb, "./naive_bayes.joblib", compress=True)
joblib.dump(lr, "./logistic_regression.joblib", compress=True)
joblib.dump(svc, "./support_vector_machine.joblib", compress=True)
joblib.dump(evc, "./voting_classifier.joblib", compress=True)

['./voting_classifier.joblib']

In [14]:
train_mode = dict(df.mode().iloc[0])
print(train_mode)

{'Gender': 'Male', 'Married': 'Yes', 'Dependents': 0.0, 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 2500.0, 'CoapplicantIncome': 0.0, 'LoanAmount': 100000, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'Property_Area': 'Semiurban', 'Loan_Status': 'Y'}
